In [ ]:
import io
import boto3
from dotenv import load_dotenv
import pandas as pd
from pathlib import Path
from ydata_profiling import ProfileReport

import sys
sys.path.append(str(Path().resolve().parent))
from utils.s3_upload import upload_dataframe_to_s3

In [ ]:
pd.options.display.max_columns = None

# DATA_PATH = Path('../data')
# RAW_DATA_PATH = DATA_PATH / 'raw'
# PROCESSED_DATA_PATH = DATA_PATH / 'processed'

# halfmarathon2023_df = pd.read_csv(RAW_DATA_PATH / 'halfmarathon_wroclaw_2023.csv', sep=';')
# halfmarathon2024_df = pd.read_csv(RAW_DATA_PATH / 'halfmarathon_wroclaw_2024.csv', sep=';')

In [ ]:

load_dotenv()

s3 = boto3.client('s3')
BUCKET_NAME = 'halfmarathon-finish-time-predictor'

files_to_load = [
    'halfmarathon_wroclaw_2023.csv',
    'halfmarathon_wroclaw_2024.csv'
]

dataframes = {}

for file_name in files_to_load:
    s3_key = f'data/raw/{file_name}'
    response = s3.get_object(Bucket=BUCKET_NAME, Key=s3_key)
    df_name = file_name.replace('.csv', '').replace('halfmarathon_wroclaw_', '')
    dataframes[f"halfmarathon{df_name}_df"] = pd.read_csv(io.BytesIO(response['Body'].read()), sep=';')

halfmarathon2023_df = dataframes['halfmarathon2023_df']
halfmarathon2024_df = dataframes['halfmarathon2024_df']

In [ ]:
halfmarathon2023_df.sample(8)

In [ ]:
halfmarathon2024_df.sample(8)

In [ ]:
halfmarathon2024_df.columns == halfmarathon2023_df.columns

In [ ]:
halfmarathon2023_df["year"] = 2023
halfmarathon2024_df["year"] = 2024

halfmarathon_df = pd.concat([halfmarathon2023_df, halfmarathon2024_df], ignore_index=True)

In [ ]:
halfmarathon_df.shape

In [ ]:
# returning_runners_df = halfmarathon_df[
#     halfmarathon_df.duplicated(subset=['Imię', 'Nazwisko']) &
#     ~(halfmarathon_df['Imię'].str.lower() == 'anonimowy') & ~(halfmarathon_df['Nazwisko'].str.lower() == 'zawodnik')
# ]

returning_runners_df = halfmarathon_df[
    ~(halfmarathon_df['Imię'].str.lower() == 'anonimowy') &
    ~(halfmarathon_df['Nazwisko'].str.lower() == 'zawodnik')
].groupby(['Imię', 'Nazwisko'])\
 .filter(lambda x: len(x) > 1)

In [ ]:
returning_runners_df[returning_runners_df['Nazwisko'] == 'ŻYTKOWSKI']
# returning_runners_df[returning_runners_df['Nazwisko'] == 'ĆWIERTNIA']
# returning_runners_df[returning_runners_df['Nazwisko'] == 'MASŁOWSKI']

In [ ]:
# ~4750 people took part in 2024 and 2025 year
returning_runners_df.shape

In [ ]:
returning_runners_df[returning_runners_df['Miejsce'].notna()].shape

In [ ]:
pd.set_option('display.max_info_columns', 50)
halfmarathon_df.info(max_cols=40)

In [ ]:
halfmarathon_df.dtypes

In [ ]:
halfmarathon_df.isnull().sum()

In [ ]:
halfmarathon_df[halfmarathon_df['20 km Tempo'].isnull()]

In [ ]:
halfmarathon_df.duplicated().sum()

In [ ]:
halfmarathon_df.describe().T

In [ ]:
halfmarathon_df['Płeć'].value_counts()

In [ ]:
halfmarathon_df['Kategoria wiekowa'].value_counts()

In [ ]:
halfmarathon_df['Kraj'].value_counts().head(10)

In [ ]:
halfmarathon_profile = ProfileReport(halfmarathon_df, title="Halfmarathon Profiling Report", explorative=True)
halfmarathon_profile.to_file("../reports/profiling/halfmarathon_data_report.html")

In [ ]:
# halfmarathon_df.to_csv(PROCESSED_DATA_PATH / 'halfmarathon_data.csv', index=False)

In [ ]:
upload_dataframe_to_s3(
    df=halfmarathon_df,
    bucket=BUCKET_NAME,
    key='data/processed/halfmarathon_data.csv'
)